In [1]:
import numpy as np
import pandas as pd

In [2]:
def place_car():
    car_pos = np.random.random()
    if car_pos <= 1/3:
        return [1,0,0]
    elif car_pos <= 2/3:
        return [0,1,0]
    else:
        return [0,0,1]

In [3]:
def play_game(pick, change, debugging=False):
    doors = place_car()
    car_position = [i for i, val in enumerate(doors) if val==1]
    not_car = [i for i, val in enumerate(doors) if val!=1]
    
    if np.random.random() <= 0.5:
        if not_car[0] != pick:
            host_open = not_car[0]
        else:
            host_open = not_car[1]
    else:
        if not_car[1] != pick:
            host_open = not_car[1]
        else:
            host_open = not_car[0]
    
    if debugging:
        print('Doors: ', doors)
        print('Host opens: ' + str(host_open))
    
    if not change:
        if debugging:
            print('Picked and stay with ' + str(pick))
        return doors[pick], host_open, car_position
    else:
        if debugging:
            chosen_door = [i for i, val in enumerate(doors) if i not in [pick, host_open]][0]
            print('Picked ' +str(pick) +' but change to ' +str(chosen_door))
        return [val for i, val in enumerate(doors) if i not in [pick, host_open]][0], host_open, car_position

#         if np.random.random() <= 0.5:
#             return doors[pick], host_open, car_position
#         else:
#             return [val for i, val in enumerate(doors) if i not in [pick, host_open]][0], host_open, car_position

#         if np.random.random() <= 0.5:
#             return doors[pick], host_open, car_position
#         else:
#             return [val for i, val in enumerate(doors) if i not in [pick]][0], host_open, car_position

In [4]:
sims = 900000

yes_change = []
no_change = []
pick_list_change = []
host_open_list_change = []
car_position_list_change = []
pick_list_no_change = []
host_open_list_no_change = []
car_position_list_no_change = []

for i in range(sims):
    pick_num = np.random.random()
    if pick_num <= 1/3:
        pick = 0
    elif pick_num <= 2/3:
        pick = 1
    else:
        pick = 2
    
    result_change, host_open_change, car_position_change = play_game(pick, change=True)
    result_no_change, host_open_no_change, car_position_no_change = play_game(pick, change=False)
    
    yes_change.append(result_change)
    pick_list_change.append(pick)
    car_position_list_change.append(car_position_change)
    host_open_list_change.append(host_open_change)
    
    no_change.append(result_no_change)
    pick_list_no_change.append(pick)
    car_position_list_no_change.append(car_position_no_change)
    host_open_list_no_change.append(host_open_no_change)

In [5]:
sum(yes_change)/sims

0.6658933333333333

In [6]:
sum(no_change)/sims

0.33318333333333333

In [7]:
change_results_df = pd.concat([pd.DataFrame(pick_list_change, 
                                            columns=['pick']),
                               pd.DataFrame(car_position_list_change, 
                                            columns=['car_position']),
                               pd.DataFrame(host_open_list_change, 
                                            columns=['host_opens']),
                               pd.DataFrame(yes_change, 
                                            columns=['result'])], 
                              axis=1)

change_results_grp = change_results_df.groupby(by=['pick',
                                                   'car_position'])\
                                      .agg(['sum','count']).reset_index()
change_results_grp['win?'] = change_results_grp['result','sum']/\
                             change_results_grp['result','count']

In [8]:
no_change_results_df = pd.concat([pd.DataFrame(pick_list_no_change, 
                                               columns=['pick']),
                                  pd.DataFrame(car_position_list_no_change, 
                                            columns=['car_position']),
                                  pd.DataFrame(host_open_list_no_change, 
                                            columns=['host_opens']),
                                  pd.DataFrame(no_change, 
                                            columns=['result'])], 
                                  axis=1)

no_change_results_grp = no_change_results_df.groupby(by=['pick',
                                                      'car_position'])\
                                         .agg(['sum','count']).reset_index()
no_change_results_grp['win?'] = no_change_results_grp['result','sum']/\
                                no_change_results_grp['result','count']

In [9]:
change_results_grp.sort_values(by=['pick','car_position'])[['pick','car_position','result','win?']]

pick car_position  result         win?
                        sum   count     
0    0            0       0  100511  0.0
1    0            1   99990   99990  1.0
2    0            2  100140  100140  1.0
3    1            0   99730   99730  1.0
4    1            1       0   99789  0.0
5    1            2   99850   99850  1.0
6    2            0   99777   99777  1.0
7    2            1   99817   99817  1.0
8    2            2       0  100396  0.0

In [10]:
no_change_results_grp.sort_values(by=['pick','car_position'])[['pick','car_position','result','win?']]

pick car_position  result         win?
                        sum   count     
0    0            0  100074  100074  1.0
1    0            1       0  100517  0.0
2    0            2       0  100050  0.0
3    1            0       0   99398  0.0
4    1            1   99866   99866  1.0
5    1            2       0  100105  0.0
6    2            0       0  100008  0.0
7    2            1       0  100057  0.0
8    2            2   99925   99925  1.0